<a href="https://colab.research.google.com/github/ishancdu/det/blob/master/Walmart_Sales_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Walmart Sales ML Project**

In [ ]:
__author__      = "Ishan Dubey"

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### **Importing relevant libraries and files**

In [ ]:
!pip install statsmodels==0.11.0
import pandas as pd 
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import plotly.express as px
import plotly
from statsmodels.tsa.seasonal import STL
from sklearn.preprocessing import OneHotEncoder

df_volume = pd.read_csv("/content/drive/My Drive/ishan_data/sales_train_evaluation.csv")
df_prices = pd.read_csv("/content/drive/My Drive/ishan_data/sell_prices.csv")
df_calendar = pd.read_csv("/content/drive/My Drive/ishan_data/calendar.csv")

'''
df_volume = pd.read_csv("sales_train_evaluation.csv")
df_prices = pd.read_csv("sell_prices.csv")
df_calendar = pd.read_csv("calendar.csv")
'''

'\ndf_volume = pd.read_csv("sales_train_evaluation.csv")\ndf_prices = pd.read_csv("sell_prices.csv")\ndf_calendar = pd.read_csv("calendar.csv")\n'

### **Adding the Volume data for all days in the given combination**

In [ ]:
df_volume['agg_volume']=0
for col in df_volume.columns[6:-1]:
    df_volume['agg_volume'] = df_volume['agg_volume'] + df_volume[col]

#creating the group dataframe
df_group = df_volume[['id','agg_volume']]
df_group = df_group.rename(columns = {'agg_volume': 'volume'})
df_volume = df_volume.drop(columns = 'agg_volume')


### **Creating the group for the Prices**

In [ ]:
#creating the group for the price
df_price_group = df_prices.groupby(['store_id', 'item_id']).agg({'sell_price':'mean'}).reset_index()
df_price_group['id'] = df_price_group['item_id'] + '_' + df_price_group['store_id'] + '_evaluation'

### **Merging the volume aggerage and the price aggregate and creating the revenue parameter**

In [ ]:
df_group_joined = pd.merge(df_group, df_price_group[['id', 'sell_price']], on= 'id', how='left')
df_group_joined['revenue'] = df_group_joined['volume'] * df_group_joined['sell_price']
df_group_joined = df_group_joined.sort_values(by='revenue',ascending=False)
print (df_group_joined)

                                  id  volume  sell_price        revenue
8442     FOODS_3_120_CA_3_evaluation   90412    4.975652  449858.664348
8412     FOODS_3_090_CA_3_evaluation  253859    1.347624  342106.509823
2344     FOODS_3_120_CA_1_evaluation   63107    4.980000  314272.860000
18055    FOODS_3_586_TX_2_evaluation  195120    1.595957  311403.217021
18639  HOBBIES_1_354_TX_3_evaluation   12049   23.191864  279438.774237
...                              ...     ...         ...            ...
9635   HOBBIES_2_073_CA_4_evaluation      45    0.537981      24.209155
9588   HOBBIES_2_026_CA_4_evaluation     100    0.230000      23.000000
9627   HOBBIES_2_065_CA_4_evaluation      38    0.500000      19.000000
9587   HOBBIES_2_025_CA_4_evaluation      28    0.500000      14.000000
12636  HOBBIES_2_025_TX_1_evaluation      28    0.500000      14.000000

[30490 rows x 4 columns]


### **Applying the Pareto ABC analysis and chosing the combinations with top 20% of sales**

In [ ]:
df_group_joined['perc'] = 0                                                                                                         
for i in range(0, df_group_joined.shape[0]):                                                                                            
    df_group_joined['perc'].iloc[i] = 100*(df_group_joined['revenue'][:i+1].sum() / df_group_joined['revenue'].sum())

#choosing the top 20% of the sales
df_top_id = df_group_joined.drop(df_group_joined[df_group_joined['perc']>20].index)
df_top_id = df_top_id.reset_index()
df_top_id

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,index,id,volume,sell_price,revenue,perc
0,8442,FOODS_3_120_CA_3_evaluation,90412,4.975652,449858.664348,0.233979
1,8412,FOODS_3_090_CA_3_evaluation,253859,1.347624,342106.509823,0.411915
2,2344,FOODS_3_120_CA_1_evaluation,63107,4.980000,314272.860000,0.575374
3,18055,FOODS_3_586_TX_2_evaluation,195120,1.595957,311403.217021,0.737340
4,18639,HOBBIES_1_354_TX_3_evaluation,12049,23.191864,279438.774237,0.882681
...,...,...,...,...,...,...
580,7112,HOUSEHOLD_1_459_CA_3_evaluation,37392,0.978310,36580.962254,19.919287
581,14702,FOODS_3_282_TX_1_evaluation,14560,2.506755,36498.352318,19.938270
582,23352,FOODS_2_183_WI_1_evaluation,4119,8.854398,36471.264974,19.957240
583,2765,FOODS_3_541_CA_1_evaluation,36420,1.000638,36443.246809,19.976194


### **Selecting the top ID from the main volume data frame and transposing it**

In [ ]:
#changing the index of the volume into the id values
df_volume.index = df_volume['id']
#selecting the dataframe with the top 20% sales ID
df_volume = df_volume.loc[df_top_id['id']]
#resetting the index
df_volume = df_volume.drop(columns = 'id').reset_index()
#transposing the data
df_volume_transpose = df_volume.melt(id_vars = ['id','item_id','cat_id','store_id','dept_id','state_id'])
df_volume_transpose=df_volume_transpose.rename(columns={'variable':'d','value':'volume'})      


## **Merging with the calendar dataframe and the sell prices data frame**

In [ ]:
#merging the calendar                                                                                                      
df_volume_transpose = pd.merge(df_volume_transpose, df_calendar,                                                                                   
                     on='d', how='left'                                                                                        
    )                                                                                                                                                                                                                                                        
#merging the prices with the main dataframe                                                                                
df_volume_transpose = pd.merge(df_volume_transpose, df_prices,                                                                                        
                     on = ['store_id','item_id',                                                                               
                         'wm_yr_wk'], how = 'left') 

In [ ]:
print(df_volume_transpose)

                                      id          item_id  ... snap_WI sell_price
0            FOODS_3_120_CA_3_evaluation      FOODS_3_120  ...       0        NaN
1            FOODS_3_090_CA_3_evaluation      FOODS_3_090  ...       0       1.25
2            FOODS_3_120_CA_1_evaluation      FOODS_3_120  ...       0        NaN
3            FOODS_3_586_TX_2_evaluation      FOODS_3_586  ...       0       1.48
4          HOBBIES_1_354_TX_3_evaluation    HOBBIES_1_354  ...       0        NaN
...                                  ...              ...  ...     ...        ...
1135480  HOUSEHOLD_1_459_CA_3_evaluation  HOUSEHOLD_1_459  ...       0       0.97
1135481      FOODS_3_282_TX_1_evaluation      FOODS_3_282  ...       0       2.56
1135482      FOODS_2_183_WI_1_evaluation      FOODS_2_183  ...       0       8.98
1135483      FOODS_3_541_CA_1_evaluation      FOODS_3_541  ...       0       1.00
1135484  HOUSEHOLD_1_334_CA_3_evaluation  HOUSEHOLD_1_334  ...       0       0.97

[1135485 rows x

## **Converting the daily data to the weekly data**

In [ ]:
# creating a dataframe of 1st day of every week
df_weekno = df_calendar[df_calendar['wday']==1][['d','wday']]
#creating the no.s for the week
df_weekno['week_no'] = [i for i in range(1,df_weekno.shape[0] +1 ) ]


## **Merging the week number with the df volume dataset and creating weekly master dataset that will be used for the analysis**

In [ ]:
df_master = pd.merge(df_volume_transpose, df_weekno[['d', 'week_no']], how = 'left', on= 'd')
#filling the nan values with the previous values
df_master['week_no'] = df_master['week_no'].ffill(axis = 0 )


## **Grouping the weekly data for the analysis**

In [ ]:
def find_unique(x):
  """
  The function is used to find the unique values in the event 
  columns
  """
  uniq_list = list(x.unique())
  uniq_list.remove(np.nan)
  if len(uniq_list) == 0:
    return np.nan
  elif len(uniq_list) > 1:
    return '-'.join(uniq_list)
  else:
    return uniq_list[0]


df_final_master = df_master.groupby(['id', 'item_id', 'cat_id', 'store_id', 'dept_id', 'state_id', 'week_no']).agg({
      'sell_price':  'mean', 'volume':'sum', 'd': 'first', 'date': 'first', 
      'month': 'first' , 'year': 'first', 'snap_CA': 'sum', 'snap_TX': 'sum', 'snap_WI': 'sum', 'event_name_1': find_unique, 
      'event_type_1': find_unique, 'event_name_2': find_unique, 'event_type_2': find_unique 
      }).reset_index()

# **Exploratory data analysis**

In [ ]:
#getting the total number of combinations
print ("Total number of unique combinations are ", len(df_final_master['id'].unique()))
print ("Total number item id ", len(df_final_master['item_id'].unique()))
print ("Total number category id ", len(df_final_master['cat_id'].unique()))
print ("Total number store id ", len(df_final_master['store_id'].unique()))
print ("Total number department id ", len(df_final_master['dept_id'].unique()))
print ("Total number states  ", len(df_final_master['state_id'].unique()))
print ("Total Holidays for event type 1 ",len(df_final_master['event_type_1'].unique()))
print ("Total Holidays for event type 2 ",len(df_final_master['event_type_2'].unique()))
print ("Total Volume Sold for all the products ", df_final_master['volume'].sum())

df_final_master[['week_no', 'sell_price', 'volume']].describe()


Total number of unique combinations are  585
Total number item id  196
Total number category id  3
Total number store id  10
Total number department id  6
Total number states   3
Total Holidays for event type 1  13
Total Holidays for event type 2  3
Total Volume Sold for all the products  12564572


,week_no,sell_price,volume
count,162630.000000,150930.000000,162630.000000
mean,139.500000,5.528958,77.258636
std,80.251415,4.483738,112.732992
min,1.000000,0.640000,0.000000
25%,70.000000,2.000000,13.000000
50%,139.500000,4.280000,41.000000
75%,209.000000,6.980000,99.000000
max,278.000000,23.980000,3539.000000


## **Demand Classification**

In [ ]:
def cv2 (x):
  return x.std()/x.mean()

def adi (x):
  return (x - x.shift(1)).mean()
  
demand_clas= df_final_master.groupby('id').agg({'volume':[cv2, 'sum'], 'week_no':adi})

df_final_master[df_final_master['id'] == 'FOODS_1_012_CA_2_evaluation'][['date','volume']]

,date,volume
0,2011-01-29,0
1,2011-02-05,0
2,2011-02-12,0
3,2011-02-19,0
4,2011-02-26,0
...,...,...
273,2016-04-23,90
274,2016-04-30,133
275,2016-05-07,81
276,2016-05-14,31


### **Scatter chart for the coeff of variation and demand signal based on the weightage of volume**


1.   Smooth Demand if cv2 < 0.49
2.   Erratic Demand if cv2 >0.49

1.   No Demand Is Intermittent or Lumpy since ADI is 1 for all the combinations





In [ ]:
fig = px.scatter(x=demand_clas[( 'volume', 'cv2')], y=demand_clas[('week_no', 'adi')],
	              size=demand_clas[( 'volume', 'sum')])
fig.update_layout(
    title = 'Demand Classification',
    shapes=[
    dict(
      yref = 'paper', y0= 0, y1= 1,
      xref = 'x', x0= 0.49, x1= 0.49
      )],
  xaxis=dict(
        title='Coefficient of Variation'
    ),
    yaxis=dict(
        title='Average Demand Interval'
    )
)
fig.show()

In [ ]:
smooth_comb = demand_clas[demand_clas[( 'volume', 'cv2')]<0.49]

#### **Total Number of smooth and erratic combaintions**

In [ ]:
print ("Total number of smooth combinations ", demand_clas[demand_clas[( 'volume', 'cv2')] < 0.49].shape[0])
print ("Volume represnted by the smooth signals ", demand_clas[demand_clas[( 'volume', 'cv2')] < 0.49][( 'volume', 'sum')].sum())
print ("Total number of erratic combinations ", demand_clas[demand_clas[( 'volume', 'cv2')] > 0.49].shape[0])
print ("Volume represnted by the erratic signals ", demand_clas[demand_clas[( 'volume', 'cv2')] > 0.49][( 'volume', 'sum')].sum())

Total number of smooth combinations  165
Volume represnted by the smooth signals  5188763
Total number of erratic combinations  420
Volume represnted by the erratic signals  7375809


## **Bar Chart for state wise sales** 




In [ ]:
store_agg = df_final_master.groupby('store_id').aggregate({'state_id':'first', 'volume': 'sum',}).reset_index()
fig = px.bar(store_agg, x='store_id', y='volume', color="state_id", title='State wise sales')
fig.show()

## **Item Category State Wise Sales**

In [ ]:
store_agg = df_final_master.groupby(['cat_id','state_id']).aggregate({'volume': 'sum',}).reset_index()
print (store_agg)
fig = px.bar(store_agg, x='state_id', y='volume', color="cat_id", title='Category State wise sales')
fig.show()

      cat_id state_id   volume
0      FOODS       CA  4947223
1      FOODS       TX  3660337
2      FOODS       WI  2703860
3    HOBBIES       CA    75457
4    HOBBIES       TX    69622
5    HOBBIES       WI    29443
6  HOUSEHOLD       CA   652319
7  HOUSEHOLD       TX   198388
8  HOUSEHOLD       WI   227923


In [ ]:
df_final_master.columns

Index(['id', 'item_id', 'cat_id', 'store_id', 'dept_id', 'state_id', 'week_no',
       'sell_price', 'volume', 'd', 'date', 'month', 'year', 'snap_CA',
       'snap_TX', 'snap_WI', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2'],
      dtype='object')

## **Graph for the total numbers of items in each department**

In [ ]:
def count_unique(x):
  return len(x.unique())

department_items = df_final_master.groupby('dept_id').agg({'item_id':count_unique}).reset_index()
fig = px.bar(department_items, x='dept_id', y='item_id', color="dept_id", title='Items in each department')
fig.show()

## **Sales in each department for each state**

In [ ]:
department_sales = df_final_master.groupby(['dept_id','state_id']).agg({'volume':'sum'}).reset_index()
fig = px.bar(department_sales, x=department_sales['dept_id']+department_sales['state_id'], y='volume', color="state_id", title='volume in each department of state')
fig.show()

## **Time Series Plot for each item category**

In [ ]:
item_cat_agg = df_final_master.groupby(['date','cat_id']).agg({'volume':'sum'}).reset_index()
fig = px.line(item_cat_agg, x="date", y="volume", color="cat_id",
              line_group="cat_id", hover_name="cat_id")
fig.show()

## **Time Series Plot for each item category**

In [ ]:
state_cat_agg = df_final_master.groupby(['date','state_id']).agg({'volume':'sum'}).reset_index()
fig = px.line(state_cat_agg, x="date", y="volume", color="state_id",
              line_group="state_id", hover_name="state_id")
fig.show()

## **Seasonality and trend decomposition for California** 

In [ ]:
subset_df = state_cat_agg[state_cat_agg['state_id']=='CA']
subset_df['date'] = pd.to_datetime(                                                           
            subset_df['date'],                                                                        
            format = '%Y-%m-%d'                                                                     
      )                     
subset_df.index = subset_df['date']
res = STL(subset_df['volume']).fit()
#creating the dataframe for the graph plotting
graph_df = res.trend.reset_index()
graph_df = pd.merge(graph_df, res.seasonal.reset_index(), on='date', how='left') 
graph_df = pd.merge(graph_df, res.resid.reset_index(), on='date', how='left')
graph_df = pd.melt(graph_df, id_vars='date')

#plotting the graph
fig = px.line(graph_df, x="date", y="value", color="variable",
              line_group="variable", hover_name="variable", title='California Seasonality and trend decomposition')
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## **Seasonality and trend decomposition for Texas** 

In [ ]:
subset_df = state_cat_agg[state_cat_agg['state_id']=='TX']
subset_df['date'] = pd.to_datetime(                                                           
            subset_df['date'],                                                                        
            format = '%Y-%m-%d'                                                                     
      )                     
subset_df.index = subset_df['date']
res = STL(subset_df['volume']).fit()
#creating the dataframe for the graph plotting
graph_df = res.trend.reset_index()
graph_df = pd.merge(graph_df, res.seasonal.reset_index(), on='date', how='left') 
graph_df = pd.merge(graph_df, res.resid.reset_index(), on='date', how='left')
graph_df = pd.melt(graph_df, id_vars='date')

#plotting the graph
fig = px.line(graph_df, x="date", y="value", color="variable",
              line_group="variable", hover_name="variable", title='Texas Seasonality and trend decomposition')
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## **Seasonality and trend decomposition for Wisconsin** 

In [ ]:
subset_df = state_cat_agg[state_cat_agg['state_id']=='WI']
subset_df['date'] = pd.to_datetime(                                                           
            subset_df['date'],                                                                        
            format = '%Y-%m-%d'                                                                     
      )                     
subset_df.index = subset_df['date']
res = STL(subset_df['volume']).fit()
#creating the dataframe for the graph plotting
graph_df = res.trend.reset_index()
graph_df = pd.merge(graph_df, res.seasonal.reset_index(), on='date', how='left') 
graph_df = pd.merge(graph_df, res.resid.reset_index(), on='date', how='left')
graph_df = pd.melt(graph_df, id_vars='date')

#plotting the graph
fig = px.line(graph_df, x="date", y="value", color="variable",
              line_group="variable", hover_name="variable", title='Wisconsin Seasonality and trend decomposition')
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## **Creating the distribution of the given time series for each state**

### Distribution for Wisconsin

In [ ]:
counts, bins = np.histogram(state_cat_agg[state_cat_agg['state_id']=='WI']['volume'], bins = 10)
bins = 0.5 * (bins[:-1] + bins[1:])
fig = px.bar(x=bins, y=counts)
fig.show()

### Distribution for Texas

In [ ]:
counts, bins = np.histogram(state_cat_agg[state_cat_agg['state_id']=='TX']['volume'], bins = 10)
bins = 0.5 * (bins[:-1] + bins[1:])
fig = px.bar(x=bins, y=counts)
fig.show()

### Distribution for California

In [ ]:
counts, bins = np.histogram(state_cat_agg[state_cat_agg['state_id']=='CA']['volume'], bins = 10)
bins = 0.5 * (bins[:-1] + bins[1:])
fig = px.bar(x=bins, y=counts)
fig.show()

## **Total Sales During the Events**

#### Sales during the Event 1

In [ ]:
df_final_master.columns

Index(['id', 'item_id', 'cat_id', 'store_id', 'dept_id', 'state_id', 'week_no',
       'sell_price', 'volume', 'd', 'date', 'month', 'year', 'snap_CA',
       'snap_TX', 'snap_WI', 'event_name_1', 'event_type_1', 'event_name_2',
       'event_type_2'],
      dtype='object')

In [ ]:
event1_agg = df_final_master.groupby(['event_name_1','state_id']).agg({'volume':'sum'}).reset_index()
fig = px.bar(
    event1_agg, x="volume", y="event_name_1", color='state_id', orientation='h',
    height=600,
    title='Sales on the event type 1'
             )
fig.show()

#### Sales during the Event 2

In [ ]:
event2_agg = df_final_master.groupby(['event_name_2','state_id']).agg({'volume':'sum'}).reset_index()

fig = px.bar(
    event2_agg, x="volume", y="event_name_2", color='state_id', orientation='h',
    height=400,
    title='Sales on the event type 2'
             )
fig.show()


## **Sales on snap week compared to other weeks**

### Sales on california snap

In [ ]:
df_final_master.columns
ca_snap_agg = df_final_master.groupby(['state_id','snap_CA']).agg({'volume':'mean'}).reset_index()
ca_snap_agg = ca_snap_agg[ca_snap_agg['state_id'] == 'CA']
fig = px.bar(
    ca_snap_agg, x="volume", y="snap_CA", orientation='h',
    height=400,
    title='Average sales on california snap weeks'
             )
fig.show()


### Sales on Texas snap

In [ ]:
df_final_master.columns
ca_snap_agg = df_final_master.groupby(['state_id','snap_TX']).agg({'volume':'mean'}).reset_index()
ca_snap_agg = ca_snap_agg[ca_snap_agg['state_id'] == 'TX']
fig = px.bar(
    ca_snap_agg, x="volume", y="snap_TX", orientation='h',
    height=400,
    title='Average sales on texas snap weeks'
             )
fig.show()

### Sales on Wisconsin snap

In [ ]:
df_final_master.columns
ca_snap_agg = df_final_master.groupby(['state_id','snap_WI']).agg({'volume':'mean'}).reset_index()
ca_snap_agg = ca_snap_agg[ca_snap_agg['state_id'] == 'WI']
fig = px.bar(
    ca_snap_agg, x="volume", y="snap_WI", orientation='h',
    height=400,
    title='Average sales on wisconsin snap weeks'
             )
fig.show()

# **Data Prepration for modelling**

## **Date features extraction**
### Features to be extracted

* **Year Number**
* **Week Number**
* **Week of Month**
* **Week of Year**



In [ ]:

#creating the week of month feature
week_of_month = df_final_master[['week_no','month','year']].groupby(['year', 'month', 'week_no']).count().reset_index()
week_of_month['week_of_month'] = 0
for grp in week_of_month.groupby(['year', 'month']):
    
    tot_week = week_of_month[(week_of_month['year']==grp[0][0]) & (week_of_month['month']==grp[0][1])].shape[0]
    ind_week = week_of_month[(week_of_month['year']==grp[0][0]) & (week_of_month['month']==grp[0][1])].index
    if len(ind_week) == 1:
        week_of_month['week_of_month'].loc[ind_week] = 1
    else:
        #pdb.set_trace()
        week_of_month['week_of_month'].loc[ind_week] = [i+1 for i in range (tot_week)] 

df_final_master = pd.merge(df_final_master, week_of_month, on=['year', 'month', 'week_no'], how='left')



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:

#Creating the week of the year
week_of_year = df_final_master[['week_no','month','year']].groupby(['year', 'month', 'week_no']).count().reset_index()
week_of_year['week_of_year'] = 0
for grp in week_of_year.groupby(['year']):
    tot_week = week_of_year[(week_of_year['year']==grp[0])].shape[0]
    ind_week = week_of_year[(week_of_year['year']==grp[0])].index
    if len(ind_week) == 1:
        week_of_year['week_of_year'].loc[ind_week] = 1
    else:
        #pdb.set_trace()
        week_of_year['week_of_year'].loc[ind_week] = [i+1 for i in range (tot_week)] 

df_final_master = pd.merge(df_final_master, week_of_year, on=['year', 'month', 'week_no'], how='left')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_final_master

,id,item_id,cat_id,store_id,dept_id,state_id,week_no,sell_price,volume,d,date,month,year,snap_CA,snap_TX,snap_WI,event_name_1,event_type_1,event_name_2,event_type_2,week_of_month,week_of_year
0,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,1.0,NaN,0,d_1,2011-01-29,1,2011,4,2,2,NaN,NaN,NaN,NaN,1,1
1,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,2.0,NaN,0,d_8,2011-02-05,2,2011,6,5,5,SuperBowl,Sporting,NaN,NaN,1,2
2,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,3.0,NaN,0,d_15,2011-02-12,2,2011,0,3,3,ValentinesDay,Cultural,NaN,NaN,2,3
3,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,4.0,NaN,0,d_22,2011-02-19,2,2011,0,0,0,PresidentsDay,National,NaN,NaN,3,4
4,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,5.0,NaN,0,d_29,2011-02-26,2,2011,4,2,2,NaN,NaN,NaN,NaN,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162625,HOUSEHOLD_2_348_CA_3_evaluation,HOUSEHOLD_2_348,HOUSEHOLD,CA_3,HOUSEHOLD_2,CA,274.0,7.44,28,d_1912,2016-04-23,4,2016,0,0,0,NaN,NaN,NaN,NaN,4,17
162626,HOUSEHOLD_2_348_CA_3_evaluation,HOUSEHOLD_2_348,HOUSEHOLD,CA_3,HOUSEHOLD_2,CA,275.0,7.44,22,d_1919,2016-04-30,4,2016,6,4,4,Pesach End-OrthodoxEaster-Cinco De Mayo,Religious-Cultural,NaN,NaN,5,18
162627,HOUSEHOLD_2_348_CA_3_evaluation,HOUSEHOLD_2_348,HOUSEHOLD,CA_3,HOUSEHOLD_2,CA,276.0,7.44,27,d_1926,2016-05-07,5,2016,4,5,4,Mother's day,Cultural,NaN,NaN,1,19
162628,HOUSEHOLD_2_348_CA_3_evaluation,HOUSEHOLD_2_348,HOUSEHOLD,CA_3,HOUSEHOLD_2,CA,277.0,7.44,12,d_1933,2016-05-14,5,2016,0,1,2,NaN,NaN,NaN,NaN,2,20


### **EDA for average sales per week of month**

In [ ]:
wom_avg_sales = df_final_master.groupby(['week_of_month', 'state_id']).agg({'volume':'mean'}).reset_index()
fig = px.bar(
    wom_avg_sales, x="volume", y="week_of_month", orientation='h',
    color = 'state_id',
    height=400,
    title='Sales per week of month'
             )
fig.show()

## **Snap and Event Features**

*   **Creation of SNAP Features**



In [ ]:
df_final_master['snap'] = 0
df_final_master['snap'].loc[
        df_final_master[df_final_master['state_id'] == 'CA']['snap_CA'].index
        ] = df_final_master['snap_CA']

df_final_master['snap'].loc[
        df_final_master[df_final_master['state_id'] == 'TX']['snap_TX'].index
        ] = df_final_master['snap_TX']

df_final_master['snap'].loc[
        df_final_master[df_final_master['state_id'] == 'WI']['snap_WI'].index
        ] = df_final_master['snap_WI']

df_final_master = df_final_master.drop(columns = ['snap_CA', 'snap_TX', 'snap_WI'])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



* **Creation of the event features**

In [ ]:
df_final_master['event'] = 0
df_final_master.at[df_final_master[df_final_master['event_name_1'].notnull()].index, 'event'] = 1
df_final_master.at[df_final_master[df_final_master['event_name_2'].notnull()].index, 'event'] = 1
df_final_master = df_final_master.drop(columns = ['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2'])
df_final_master

,id,item_id,cat_id,store_id,dept_id,state_id,week_no,sell_price,volume,d,date,month,year,week_of_month,week_of_year,snap,event
0,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,1.0,NaN,0,d_1,2011-01-29,1,2011,1,1,4,0
1,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,2.0,NaN,0,d_8,2011-02-05,2,2011,1,2,6,1
2,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,3.0,NaN,0,d_15,2011-02-12,2,2011,2,3,0,1
3,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,4.0,NaN,0,d_22,2011-02-19,2,2011,3,4,0,1
4,FOODS_1_012_CA_2_evaluation,FOODS_1_012,FOODS,CA_2,FOODS_1,CA,5.0,NaN,0,d_29,2011-02-26,2,2011,4,5,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162625,HOUSEHOLD_2_348_CA_3_evaluation,HOUSEHOLD_2_348,HOUSEHOLD,CA_3,HOUSEHOLD_2,CA,274.0,7.44,28,d_1912,2016-04-23,4,2016,4,17,0,0
162626,HOUSEHOLD_2_348_CA_3_evaluation,HOUSEHOLD_2_348,HOUSEHOLD,CA_3,HOUSEHOLD_2,CA,275.0,7.44,22,d_1919,2016-04-30,4,2016,5,18,6,1
162627,HOUSEHOLD_2_348_CA_3_evaluation,HOUSEHOLD_2_348,HOUSEHOLD,CA_3,HOUSEHOLD_2,CA,276.0,7.44,27,d_1926,2016-05-07,5,2016,1,19,4,1
162628,HOUSEHOLD_2_348_CA_3_evaluation,HOUSEHOLD_2_348,HOUSEHOLD,CA_3,HOUSEHOLD_2,CA,277.0,7.44,12,d_1933,2016-05-14,5,2016,2,20,0,0


## **Imputing the NaN Values in the price**

In [ ]:
df_final_master['sell_price'] = df_final_master['sell_price'].ffill()
df_final_master['sell_price'] = df_final_master['sell_price'].bfill()

## **Calculation of lag features**


In [ ]:
def lag_func(df):                                                                                                              
    df['vol_lag1'] = df['volume'].shift(1)                                                                                       
    df['vol_lag2'] = df['volume'].shift(2)                                                                                       
    df['vol_lag3'] = df['volume'].shift(3)                                                                                       
    df['vol_lag4'] = df['volume'].shift(4)     
    return df

lag_df = df_final_master.groupby('id').apply(lag_func)
lag_df = lag_df.dropna()



## **Creation of categorical features**



In [ ]:
dummies = pd.get_dummies(lag_df, columns= ['id'])

#removing the columns to remove perfect multicollinearity and the other category variables
#and creating the modelling dataframe
modelling_df = dummies.drop(columns = ['id_FOODS_1_012_CA_2_evaluation',
                                      'store_id', 'cat_id', 'dept_id', 'state_id', 
                                      'd', 'date', 'item_id'])

modelling_df

,week_no,sell_price,volume,month,year,week_of_month,week_of_year,snap,event,vol_lag1,vol_lag2,vol_lag3,vol_lag4,id_FOODS_1_012_CA_3_evaluation,id_FOODS_1_012_CA_4_evaluation,id_FOODS_1_012_TX_1_evaluation,id_FOODS_1_012_TX_2_evaluation,id_FOODS_1_012_TX_3_evaluation,id_FOODS_1_043_CA_3_evaluation,id_FOODS_1_055_CA_2_evaluation,id_FOODS_1_055_CA_3_evaluation,id_FOODS_1_087_TX_2_evaluation,id_FOODS_1_087_TX_3_evaluation,id_FOODS_1_096_TX_2_evaluation,id_FOODS_1_096_TX_3_evaluation,id_FOODS_1_096_WI_1_evaluation,id_FOODS_1_096_WI_2_evaluation,id_FOODS_1_096_WI_3_evaluation,id_FOODS_1_129_TX_2_evaluation,id_FOODS_1_129_TX_3_evaluation,id_FOODS_1_129_WI_2_evaluation,id_FOODS_1_150_CA_2_evaluation,id_FOODS_1_170_CA_2_evaluation,id_FOODS_1_170_CA_3_evaluation,id_FOODS_1_170_CA_4_evaluation,id_FOODS_1_170_TX_2_evaluation,id_FOODS_1_170_TX_3_evaluation,id_FOODS_1_180_CA_3_evaluation,id_FOODS_1_187_CA_2_evaluation,id_FOODS_1_187_CA_3_evaluation,...,id_HOUSEHOLD_1_409_CA_3_evaluation,id_HOUSEHOLD_1_409_WI_2_evaluation,id_HOUSEHOLD_1_413_CA_1_evaluation,id_HOUSEHOLD_1_413_CA_2_evaluation,id_HOUSEHOLD_1_413_CA_3_evaluation,id_HOUSEHOLD_1_416_CA_3_evaluation,id_HOUSEHOLD_1_427_WI_2_evaluation,id_HOUSEHOLD_1_427_WI_3_evaluation,id_HOUSEHOLD_1_434_CA_3_evaluation,id_HOUSEHOLD_1_440_CA_1_evaluation,id_HOUSEHOLD_1_440_CA_2_evaluation,id_HOUSEHOLD_1_440_CA_3_evaluation,id_HOUSEHOLD_1_440_TX_2_evaluation,id_HOUSEHOLD_1_440_WI_3_evaluation,id_HOUSEHOLD_1_441_WI_3_evaluation,id_HOUSEHOLD_1_449_TX_3_evaluation,id_HOUSEHOLD_1_459_CA_3_evaluation,id_HOUSEHOLD_1_469_CA_2_evaluation,id_HOUSEHOLD_1_469_CA_3_evaluation,id_HOUSEHOLD_1_469_TX_1_evaluation,id_HOUSEHOLD_1_469_TX_2_evaluation,id_HOUSEHOLD_1_494_CA_1_evaluation,id_HOUSEHOLD_1_494_CA_3_evaluation,id_HOUSEHOLD_1_494_TX_1_evaluation,id_HOUSEHOLD_1_497_CA_3_evaluation,id_HOUSEHOLD_1_497_TX_2_evaluation,id_HOUSEHOLD_1_497_TX_3_evaluation,id_HOUSEHOLD_1_506_CA_3_evaluation,id_HOUSEHOLD_1_514_CA_3_evaluation,id_HOUSEHOLD_1_514_WI_2_evaluation,id_HOUSEHOLD_1_535_CA_1_evaluation,id_HOUSEHOLD_1_535_CA_2_evaluation,id_HOUSEHOLD_1_535_CA_3_evaluation,id_HOUSEHOLD_1_535_TX_2_evaluation,id_HOUSEHOLD_1_535_TX_3_evaluation,id_HOUSEHOLD_1_535_WI_3_evaluation,id_HOUSEHOLD_1_537_CA_2_evaluation,id_HOUSEHOLD_1_537_CA_3_evaluation,id_HOUSEHOLD_1_541_CA_3_evaluation,id_HOUSEHOLD_2_348_CA_3_evaluation
4,5.0,6.08,0,2,2011,4,5,4,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,6.0,6.08,0,3,2011,1,6,6,1,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,7.0,6.08,0,3,2011,2,7,0,1,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,8.0,6.08,0,3,2011,3,8,0,1,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,9.0,6.08,0,3,2011,4,9,1,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162625,274.0,7.44,28,4,2016,4,17,0,0,23.0,37.0,6.0,33.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
162626,275.0,7.44,22,4,2016,5,18,6,1,28.0,23.0,37.0,6.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
162627,276.0,7.44,27,

# **Creation of training and testing dataframes**

In [ ]:
div= 265
#div= 274
test_df_x = modelling_df[modelling_df['week_no'] > div].drop(columns ='volume')                                                                                                      
test_df_y = modelling_df[modelling_df['week_no'] > div]['volume']                                                                                                                    
                                                                                                                               
train_df_x = modelling_df[modelling_df['week_no'] <= div].drop(columns ='volume')
train_df_y = modelling_df[modelling_df['week_no'] <= div]['volume']                                                                                                            

test_df = modelling_df[modelling_df['week_no'] > div]     

# **Running the Machine learning models and checking the results on the test dataset**
## We will run the following models

*   Multillinear Regression 
*   Random Forest
*   K-nearest neighbours(KNN)








### Mape Function

In [ ]:
import math
def calc_mape(act, forc):                                                                                                                                                                                                                             
    num = abs((np.array(act)-np.array(forc)))                                                                                  
    den = abs(np.array(act))                                                                                                   
                                                                                                                             
    #replacing the infinity values as 0                                                                                        
    res = num/den          
    res = [i for i in res if math.isnan(i) == False]  
    res = [i for i in res if i != float('inf')] 
    return np.mean(res) * 100   

## **Linear Regression**

In [ ]:
#multilinear regression model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

#training the model
model_lin_reg = LinearRegression().fit(train_df_x, train_df_y)

#calculating the results on the train set and test set 
#the calculated metrics are MAPE and MSE
train_reg_mape = calc_mape(train_df_y.values, model_lin_reg.predict(train_df_x))
test_reg_mape = calc_mape(test_df_y.values, model_lin_reg.predict(test_df_x))

train_reg_mse = mean_squared_error(train_df_y.values, model_lin_reg.predict(train_df_x))
test_reg_mse = mean_squared_error(test_df_y.values, model_lin_reg.predict(test_df_x))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

divide by zero encountered in true_divide



## **Random Forest Regression**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#training the random forest model based on the best tuned hyperparameters using grid search
rf_model = RandomForestRegressor(verbose = 2, n_jobs = -1, 
                                 n_estimators = 40, max_depth = None
                                 ).fit(train_df_x, train_df_y)

#calculating the results on the train set and test set 
#the calculated metrics are MAPE and MSE
train_rf_mape = calc_mape(train_df_y.values, rf_model.predict(train_df_x))
test_rf_mape = calc_mape(test_df_y.values, rf_model.predict(test_df_x))

train_rf_mse = mean_squared_error(train_df_y.values, rf_model.predict(train_df_x))
test_rf_mse = mean_squared_error(test_df_y.values, rf_model.predict(test_df_x))


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 40
building tree 2 of 40
building tree 3 of 40
building tree 4 of 40
building tree 5 of 40
building tree 6 of 40
building tree 7 of 40
building tree 8 of 40
building tree 9 of 40
building tree 10 of 40
building tree 11 of 40
building tree 12 of 40
building tree 13 of 40
building tree 14 of 40
building tree 15 of 40
building tree 16 of 40
building tree 17 of 40
building tree 18 of 40
building tree 19 of 40
building tree 20 of 40
building tree 21 of 40
building tree 22 of 40
building tree 23 of 40
building tree 24 of 40
building tree 25 of 40
building tree 26 of 40
building tree 27 of 40
building tree 28 of 40
building tree 29 of 40
building tree 30 of 40
building tree 31 of 40
building tree 32 of 40
building tree 33 of 40
building tree 34 of 40
building tree 35 of 40
building tree 36 of 40
building tree 37 of 40
building tree 38 of 40


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.7min


building tree 39 of 40
building tree 40 of 40


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:  3.9min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    2.0s
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    2.1s finished
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in true_divide

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    0.1s finished
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in true_divide

[Parallel(n_j

In [ ]:
smooth_comb.index
import pdb
test_smooth = pd.DataFrame()
for col in smooth_comb.index:
    wdf = test_df[test_df['id_'+col] == 1]
    test_smooth = pd.concat([test_smooth, wdf])

In [ ]:
calc_mape(test_smooth['volume'], rf_model.predict(test_smooth.drop(columns = 'volume')))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done  40 out of  40 | elapsed:    0.1s finished
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

divide by zero encountered in true_divide

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

invalid value encountered in true_divide



34.25322554636123

In [ ]:
calc_mape(test_smooth['volume'], model_lin_reg.predict(test_smooth.drop(columns = 'volume')))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

divide by zero encountered in true_divide



32.982413302024995

## **K Nearest Neighbors Regression**

# **Graphs for the MAPE and MSE accuracy**

In [ ]:
dic = {'Algorithm': ['Regression', 'Regression', 'Random Forest', 'Random Forest', 'KNN', 'KNN'],
       'Type': ['Train', 'Test', 'Train', 'Test', 'Train', 'Test'],
       'MAPE': [train_reg_mape, test_reg_mape, train_rf_mape, test_rf_mape, 40.81977133542074, 54.022266633198676],
       'MSE': [train_reg_mse, test_reg_mse, train_rf_mse, test_rf_mse, 1184.3638599731473, 1433.2444181459564]
       }
results = pd.DataFrame(dic)
fig = px.bar(results, x="Algorithm", color="Type",
             y='MAPE',
             title="MAPE Results for each model",
             barmode='group',
             height=600
            )
fig.show()
fig2 = px.bar(results, x="Algorithm", color="Type",
             y='MSE',
             title="MSE Results for each model",
             barmode='group',
             height=600
            )
fig2.show()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

#running the model
knn_model = KNeighborsRegressor(n_neighbors=5).fit(train_df_x, train_df_y)


#calculating the results on the train set and test set 
#the calculated metrics are MAPE and MSE
train_knn_mape = calc_mape(train_df_y.values, knn_model.predict(train_df_x))
test_knn_mape = calc_mape(test_df_y.values, knn_model.predict(test_df_x))

train_knn_mse = mean_squared_error(train_df_y.values, knn_model.predict(train_df_x))
test_knn_mse = mean_squared_error(test_df_y.values, knn_model.predict(test_df_x))